In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
from wordcloud import WordCloud
from textwrap import wrap
import unicodedata


# "..\Data\Banco-Preguntas-Icfes.txt"

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\santi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\santi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
file_path = "../Data/Banco-Preguntas-Icfes.txt"

# Leer el archivo línea por línea
with open(file_path, "r", encoding="utf-8") as file:
    lines = file.readlines()

# Mostrar las primeras 10 líneas para verificar el formato
print("Contenido del archivo (primeras 10 líneas):\n")
for i, line in enumerate(lines[:10]):
    print(f"Línea {i+1}: {line.strip()}")


Contenido del archivo (primeras 10 líneas):

Línea 1: Pregunta: ¿En qué fechas se dará inicio al proceso de registro e inscripción a los exámenes?
Línea 2: Respuesta: Validar en la página www.icfes.gov.co / Evaluaciones y Elija el examen de su interés.
Línea 3: 
Línea 4: Pregunta:¿Cómo puedo cambiar el correo de notificaciones asociado al usuario PRISMA de mi sede jornada?
Línea 5: Respuesta:A través del Chat, ingresando a la  página www.icfes.gov.co / Atención y servicios a la ciudadanía / canales de atención / Chat.(Link:https://www.icfes.gov.co/atencion-y-servicios-a-la-ciudadania/canales-de-atencion/)
Línea 6: 
Línea 7: Pregunta:¿qué debo hacer para presentar las pruebas icfes por segunda vez?
Línea 8: Respuesta:Se puede registrar las veces que desee, después de la quinta inscripción la tarifa aumenta, validar en  la página www.icfes.gov.co / Evaluaciones y Elija el examen de su interés.
Línea 9: 
Línea 10: Pregunta:¿El examen saber 11°, entrega certificado o constancia de asistenc

In [15]:
# 📌 1️⃣ Inicializar listas para almacenar preguntas y respuestas
preguntas = []
respuestas = []
pregunta_actual = ""
respuesta_actual = ""
leyendo_respuesta = False

# 📌 2️⃣ Leer el archivo línea por línea y separar preguntas y respuestas
with open(file_path, "r", encoding="utf-8") as file:
    for line in file:
        line = line.strip()

        # Detectar preguntas
        if line.startswith("Pregunta:"):
            if pregunta_actual:
                preguntas.append(pregunta_actual.strip())
                respuestas.append(respuesta_actual.strip())
            pregunta_actual = line.replace("Pregunta:", "").strip()
            respuesta_actual = ""
            leyendo_respuesta = False

        # Detectar respuestas
        elif line.startswith("Respuesta:"):
            leyendo_respuesta = True
            respuesta_actual = line.replace("Respuesta:", "").strip()

        # Agregar contenido a la pregunta o respuesta
        else:
            if leyendo_respuesta:
                respuesta_actual += " " + line.strip()
            else:
                pregunta_actual += " " + line.strip()

# 📌 3️⃣ Agregar la última pregunta y respuesta
if pregunta_actual:
    preguntas.append(pregunta_actual.strip())
    respuestas.append(respuesta_actual.strip())

# 📌 4️⃣ Crear DataFrames separados para preguntas y respuestas
df_preguntas = pd.DataFrame({"Pregunta": preguntas})
df_respuestas = pd.DataFrame({"Respuesta": respuestas})

# 📌 5️⃣ Mostrar las primeras filas de cada DataFrame
print("\n📊 Primeras preguntas extraídas:")
print(df_preguntas.head())

print("\n📊 Primeras respuestas extraídas:")
print(df_respuestas.head())

# 📌 6️⃣ Mostrar el número total de preguntas y respuestas extraídas
print(f"\nTotal de preguntas extraídas: {len(df_preguntas)}")
print(f"Total de respuestas extraídas: {len(df_respuestas)}")


📊 Primeras preguntas extraídas:
                                            Pregunta
0  ¿En qué fechas se dará inicio al proceso de re...
1  ¿Cómo puedo cambiar el correo de notificacione...
2  ¿qué debo hacer para presentar las pruebas icf...
3  ¿El examen saber 11°, entrega certificado o co...
4                       ¿Debo pagar las pruebas TyT?

📊 Primeras respuestas extraídas:
                                           Respuesta
0  Validar en la página www.icfes.gov.co / Evalua...
1  A través del Chat, ingresando a la  página www...
2  Se puede registrar las veces que desee, despué...
3  No, únicamente se genera certificado de asiste...
4  Valida las tarifas en la página www.icfes.gov....

Total de preguntas extraídas: 26
Total de respuestas extraídas: 26


# Punto critico de limpieza, se borra los caracteres especiales, tildes, numeros y se pasa todo a minusculas
Nota: para entregar los datos al RAG se puesden guardar un mapeo de los datos originales y el modelo trabaje con los datos limpios, y antes de entregar la respuesta al usuario busque en el mapeo el dato original y lo devuelva al usuario

```python

In [ ]:
# 📌 1️⃣ Función para limpiar el texto
def limpiar_texto(texto):
    if pd.isna(texto):  # Verificar si el texto es NaN o None
        return ""
    
    texto = str(texto).lower()  # Convertir a minúsculas
    texto = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('utf-8')  # Eliminar tildes
    texto = re.sub(r'\d+', '', texto)  # Eliminar números
    texto = re.sub(r'[^\w\s]', '', texto)  # Eliminar caracteres especiales
    texto = re.sub(r'\b[a-zA-Z]\b', '', texto)  # Eliminar palabras de 1 solo carácter
    texto = re.sub(r'\s+', ' ', texto).strip()  # Eliminar espacios extra
    
    return texto

# 📌 2️⃣ Aplicar la función de limpieza a los DataFrames
df_preguntas["Pregunta"] = df_preguntas["Pregunta"].apply(limpiar_texto)
df_respuestas["Respuesta"] = df_respuestas["Respuesta"].apply(limpiar_texto)

# 📌 3️⃣ Mostrar las primeras filas de los DataFrames limpios
print("\nPrimeras preguntas limpias:")
print(df_preguntas.head())

print("\nPrimeras respuestas limpias:")
print(df_respuestas.head())



📊 Primeras preguntas limpias:
                                            Pregunta
0  en que fechas se dara inicio al proceso de reg...
1  como puedo cambiar el correo de notificaciones...
2  que debo hacer para presentar las pruebas icfe...
3  el examen saber entrega certificado constancia...
4                         debo pagar las pruebas tyt

📊 Primeras respuestas limpias:
                                           Respuesta
0  validar en la pagina wwwicfesgovco evaluacione...
1  traves del chat ingresando la pagina wwwicfesg...
2  se puede registrar las veces que desee despues...
3  no unicamente se genera certificado de asisten...
4  valida las tarifas en la pagina wwwicfesgovco ...


In [17]:
# 📌 2️⃣ Definir stopwords en español
stop_words = set(stopwords.words("spanish"))

# 📌 3️⃣ Función para tokenizar y limpiar palabras
def tokenizar_y_filtrar(texto):
    tokens = word_tokenize(texto)  # Tokenizar el texto
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]  # Filtrar stopwords y símbolos
    return tokens

# 📌 4️⃣ Tokenizar preguntas y respuestas
tokens_preguntas = [token for pregunta in df_preguntas["Pregunta"] for token in tokenizar_y_filtrar(pregunta)]
tokens_respuestas = [token for respuesta in df_respuestas["Respuesta"] for token in tokenizar_y_filtrar(respuesta)]

# 📌 5️⃣ Contar la frecuencia de palabras
frecuencia_preguntas = Counter(tokens_preguntas)
frecuencia_respuestas = Counter(tokens_respuestas)

# 📌 6️⃣ Obtener las 10 palabras más comunes
top_preguntas = frecuencia_preguntas.most_common(10)
top_respuestas = frecuencia_respuestas.most_common(10)

# 📌 7️⃣ Función para graficar palabras más comunes
def graficar_palabras_comunes(palabras_comunes, titulo):
    palabras, frecuencias = zip(*palabras_comunes)
    plt.figure(figsize=(10, 5))
    sns.barplot(x=list(frecuencias), y=list(palabras), palette="viridis")
    plt.title(titulo)
    plt.xlabel("Frecuencia")
    plt.ylabel("Palabras")
    plt.show()

# 📌 8️⃣ Generar gráficos para preguntas y respuestas
graficar_palabras_comunes(top_preguntas, "Palabras Más Comunes en Preguntas")
graficar_palabras_comunes(top_respuestas, "Palabras Más Comunes en Respuestas")

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\santi/nltk_data'
    - 'c:\\Users\\santi\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\Local\\pypoetry\\Cache\\virtualenvs\\icfes_pro-tNgj2KN3-py3.12\\nltk_data'
    - 'c:\\Users\\santi\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\Local\\pypoetry\\Cache\\virtualenvs\\icfes_pro-tNgj2KN3-py3.12\\share\\nltk_data'
    - 'c:\\Users\\santi\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\Local\\pypoetry\\Cache\\virtualenvs\\icfes_pro-tNgj2KN3-py3.12\\lib\\nltk_data'
 